In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns

pd.__version__
xr.__version__
np.__version__
sns.__version__

'0.11.2'

In [43]:
data = pd.read_pickle("all.pkl")

# R package for SE of stats

In [2]:
import os
os.environ["R_HOME"] = "C:\\Program Files\\R\\R-4.1.3\\"

In [3]:
import os
import rpy2
from rpy2.robjects import r
from rpy2.robjects.packages import importr

In [4]:
%load_ext rpy2.ipython

C:\ProgramData\Anaconda3\envs\q2022\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [5]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [6]:
#import pandas as pd
#import rpy2.robjects as ro
#from rpy2.robjects.conversion import localconverter
#%Rdevice png
#import rpy2.ipython.html
#rpy2.ipython.html.init_printing()

In [7]:
pandas2ri

<module 'rpy2.robjects.pandas2ri' from 'C:\\ProgramData\\Anaconda3\\envs\\q2022\\lib\\site-packages\\rpy2\\robjects\\pandas2ri.py'>

In [8]:
import rpy2.situation
[print(row) for row in rpy2.situation.iter_info()];

rpy2 version:
3.4.5
Python version:
3.9.11 (main, Mar 30 2022, 02:45:55) [MSC v.1916 64 bit (AMD64)]
Looking for R's HOME:
    Environment variable R_HOME: C:\Program Files\R\R-4.1.3\
    InstallPath in the registry: C:\Program Files\R\R-4.1.3
    Environment variable R_USER: None
    Environment variable R_LIBS_USER: None
R version:
    In the PATH: 
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:


In [9]:
%R .libPaths()
%R R.home()
%R Sys.getenv('R_HOME')

'C:/Program Files/R/R-4.1.3/library'


'C:/Program Files/R/R-4.1.3/'


'C:/Program Files/R/R-4.1.3/'


# Py

In [44]:
import performanceanalytics.statistics as pas
import pandas as pd
import matplotlib.pyplot as plt

In [61]:
data = pd.read_csv("nz.csv", index_col=0, parse_dates=True)
data

,Fund Return,T-Bills,Benchmark
Date,,,
2003-09-30,0.0000,0.0000,0.0000
2003-10-31,0.0039,0.0042,0.0074
2003-11-30,0.0073,0.0041,0.0025
2003-12-31,0.0155,0.0044,0.0173
2004-01-31,0.0047,0.0041,0.0064
...,...,...,...
2018-07-31,0.0202,0.0015,0.0206
2018-08-31,0.0107,0.0014,0.0152
2018-09-30,0.0045,0.0013,-0.0009


In [62]:
pas.vol(data)

Fund Return    0.026127
T-Bills        0.001790
Benchmark      0.027417
dtype: float64

In [65]:
import performanceanalytics.table.table as pat
pat.stats_table(
    data,
    manager_col=[0],
    other_cols=[2]
)

KeyError: [['Fund Return']]

In [83]:
%%R
StdDev_ = function (R, ..., clean = c("none", "boudt", "geltner", "locScaleRob"), 
    portfolio_method = c("single", "component"), weights = NULL, 
    mu = NULL, sigma = NULL, use = "everything", method = c("pearson", 
        "kendall", "spearman"), SE = FALSE, SE.control = NULL) 
{
    if (SE) {
        if (is.null(SE.control)) 
            SE.control <- RPESE.control(estimator = "SD")
        portfolio_method = "single"
        if (SE.control$cleanOutliers == "locScaleRob") 
            clean = "locScaleRob"
        else clean = "none"
    }
    portfolio_method = portfolio_method[1]
    clean = clean[1]
    R <- checkData(R, method = "xts", ...)
    columns = colnames(R)
    if (is.null(weights) & portfolio_method != "single") {
        message("no weights passed in, assuming equal weighted portfolio")
        weights = t(rep(1/dim(R)[[2]], dim(R)[[2]]))
    }
    if (!is.null(weights)) {
        if (is.vector(weights)) {
            if (length(weights) != ncol(R)) {
                stop("number of items in weighting vector not equal to number of columns in R")
            }
        }
        else {
            weights = checkData(weights, method = "matrix", ...)
            if (ncol(weights) != ncol(R)) {
                stop("number of columns in weighting timeseries not equal to number of columns in R")
            }
        }
    }
    if (clean != "none") {
        R = as.matrix(Return.clean(R, method = clean))
    }
    if (isTRUE(SE)) {
        if (!requireNamespace("RPESE", quietly = TRUE)) {
            stop("Package \"pkg\" needed for standard errors computation. Please install it.", 
                call. = FALSE)
        }
        if (portfolio_method != "single") 
            stop("For SE computation, the portfolio method must be \"single\" to return an output.")
        ses = list()
        for (mymethod in SE.control$se.method) {
            ses[[mymethod]] = RPESE::EstimatorSE(R, estimator.fun = "SD", 
                se.method = mymethod, cleanOutliers = SE.control$cleanOutliers, 
                fitting.method = SE.control$fitting.method, freq.include = SE.control$freq.include, 
                freq.par = SE.control$freq.par, a = SE.control$a, 
                b = SE.control$b)$se
        }
        ses <- t(data.frame(ses))
    }
    switch(portfolio_method, single = {
        if (is.null(weights)) {
            tsd = t(sd(R, na.rm = TRUE))
            rownames(tsd) <- "StdDev"
        } else {
            if (!hasArg(sigma) | is.null(sigma)) sigma = cov(R, 
                use = use, method = method[1])
            tsd <- StdDev.MM(w = weights, sigma = sigma)
        }
        if (SE) return(rbind(tsd, ses)) else return(tsd)
    }, component = {
        weights = as.vector(weights)
        names(weights) <- colnames(R)
        if (is.null(sigma)) {
            sigma = cov(R, use = use, method = method[1])
        }
        return(Portsd(w = weights, sigma))
    })
}

In [74]:
%%R

library(PerformanceAnalytics)
library(RPESE)
library(xts) # Need this for the next line and later use of plot.zoo

data(edhec, package='PerformanceAnalytics')
names(edhec) <- c("CA", "CTA", "DIS", "EM","EMN", "ED", "FIA", "GM", "LS","MA", "RV", "SS", "FOF")

print(class(edhec))
print(names(edhec))
print(range(index(edhec)))

[1] "xts" "zoo"
 [1] "CA"  "CTA" "DIS" "EM"  "EMN" "ED"  "FIA" "GM"  "LS"  "MA"  "RV"  "SS" 
[13] "FOF"
[1] "1997-01-31" "2019-11-30"


In [86]:
%R StdDev_(edhec$CA, SE=F)#TRUE)

R[write to console]: Error in sd.xts(R, na.rm = TRUE) : could not find function "sd.xts"




Error in sd.xts(R, na.rm = TRUE) : could not find function "sd.xts"


RInterpreterError: Failed to parse and evaluate line 'StdDev_(edhec$CA, SE=F)#TRUE)'.
R error message: 'Error in sd.xts(R, na.rm = TRUE) : could not find function "sd.xts"'

In [85]:
%%R

StdDev_(edhec$CA, SE=F)#TRUE)

#ESout.CA = t(StdDev(edhec$CA, SE=TRUE))
#ESout.CA

SE.control <- RPESE.control(estimator = "SD")
SE.control
R = edhec$CA
res = RPESE::EstimatorSE(
    R, 
    estimator.fun = "SD", 
    se.method = "IFiid", 
    cleanOutliers = SE.control$cleanOutliers, 
    fitting.method = SE.control$fitting.method, 
    freq.include = SE.control$freq.include, 
    freq.par = SE.control$freq.par, 
    a = SE.control$a, 
    b = SE.control$b
)$se



R[write to console]: Error in sd.xts(R, na.rm = TRUE) : could not find function "sd.xts"




Error in sd.xts(R, na.rm = TRUE) : could not find function "sd.xts"


RInterpreterError: Failed to parse and evaluate line '\nStdDev_(edhec$CA, SE=F)#TRUE)\n\n#ESout.CA = t(StdDev(edhec$CA, SE=TRUE))\n#ESout.CA\n\nSE.control <- RPESE.control(estimator = "SD")\nSE.control\nR = edhec$CA\nres = RPESE::EstimatorSE(\n    R, \n    estimator.fun = "SD", \n    se.method = "IFiid", \n    cleanOutliers = SE.control$cleanOutliers, \n    fitting.method = SE.control$fitting.method, \n    freq.include = SE.control$freq.include, \n    freq.par = SE.control$freq.par, \n    a = SE.control$a, \n    b = SE.control$b\n)$se\n\n'.
R error message: 'Error in sd.xts(R, na.rm = TRUE) : could not find function "sd.xts"'

In [20]:
%%R
ES(edhec$CA, SE=TRUE)

R[write to console]: Error in (function (..., row.names = NULL, check.rows = FALSE, check.names = TRUE,  : 
  arguments imply differing number of rows: 1, 0




Error in (function (..., row.names = NULL, check.rows = FALSE, check.names = TRUE,  : 
  arguments imply differing number of rows: 1, 0


RInterpreterError: Failed to parse and evaluate line 'ES(edhec$CA, SE=TRUE)\n'.
R error message: 'Error in (function (..., row.names = NULL, check.rows = FALSE, check.names = TRUE,  : \n  arguments imply differing number of rows: 1, 0'